In [1]:
import queue
import threading
import time
import socket
import sys 
import pickle
import os

In [2]:
class myThread (threading.Thread):
    def __init__(self, threadID, name, conn, query, subServerPort):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.conn = conn
        self.query = query
        self.subServerPort = subServerPort
    def run(self):
        print( "Starting " + self.name)
        mergedResult = []
        for server in self.subServerPort:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

            s.connect(("localhost", server))
            s.send(pickle.dumps(query))
            res = pickle.loads(s.recv(1024))
            print(server, " result: ",res)
            if res != "file Not Found":
                mergedResult.append(res)
            s.close()

            
        print(mergedResult)
        if len(mergedResult):
            self.conn.send(pickle.dumps(mergedResult))
        else:
            self.conn.send(pickle.dumps("file Not Found"))
    
        self.conn.close()
        print ("Exiting " + self.name)



In [ ]:
subserver = [12300, 12301, 12302, 12303, 12304]
threads=[]
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# 12345 mainServer port
s.bind(('localhost', 12345))
s.listen(5)     
print ("MainServer socket is listening")
for i in range(5):
    c, addr = s.accept()
    print ('Got connection from', addr )
    query = pickle.loads(c.recv(1000))
#     query = "_SubServer00File_00.txt"
    thread = myThread(i+1, str("Thread-"+ str(i+1)), c, query, subserver)
    thread.start()
    threads.append(thread)
    
for x in threads:
    x.join()
print("exiting main thread")

MainServer socket is listening
